<a href="https://colab.research.google.com/github/romenlaw/NaiveNeuralNetwork/blob/main/makemore_backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Makemore - backprop ninja

## prepare datasets

In [1]:
!curl -O https://raw.githubusercontent.com/romenlaw/NaiveNeuralNetwork/main/names.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  222k  100  222k    0     0   431k      0 --:--:-- --:--:-- --:--:--  431k


In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
len(words), max(len(w) for w in words), words[:8]

(32033,
 15,
 ['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia'])

In [4]:
vocab = sorted(list(set(''.join(words))))
vocab.insert(0, '.')
itos = { i:s for i,s in enumerate(vocab)}
stoi = { s:i for i,s in enumerate(vocab)}
vocab_size = len(vocab)  # 27

In [5]:
block_size = 3  # context size - 3 tokens

def build_dataset(words):
  """returns torch tensors X, Y where
  X is a list of n-grams indices covering the whole words list, where n=block_size
  Y is a list of indices predicting each n-gram in X
  """
  X, Y = [], []

  #for w in words[:5]:
  for w in words:
    context = [0] * block_size # repeat '.' to fill block_size
    for ch in w+'.':
      ix = stoi[ch]
      #print(' '.join([itos[i] for i in context]), '---->', itos[ix])
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]

  return torch.tensor(X), torch.tensor(Y)

X, Y = build_dataset(words)
#X[:32], Y[:32]

# split the data into 3 sets
# 80% for training set
# 10% for validation/development
# 10% for testing
import random
random.seed(42)
n1 = int(len(words) * .8)
n2 = int(len(words) * .9)
random.shuffle(words) # shuffle is in-place
X_train, Y_train = build_dataset(words[:n1])
X_dev, Y_dev = build_dataset(words[n1:n2])
X_test, Y_test = build_dataset(words[n2:])

#len(words[n1:n2])
(X_train.shape, Y_train.shape), (X_dev.shape, Y_dev.shape), (X_test.shape, Y_test.shape)

((torch.Size([182625, 3]), torch.Size([182625])),
 (torch.Size([22655, 3]), torch.Size([22655])),
 (torch.Size([22866, 3]), torch.Size([22866])))

## utilities

In [6]:
# utility to compare our manual gradients with pytorch gradients
def cmp(s, dt, t):
  """Compares dt and t.grad to see if their values are equal or close
  s - name of the parameter being compared, used in printing only
  dt - tensor of manually calculated gradient
  t - torch tensor
  """
  ex = torch.all(dt==t.grad).item()
  apx = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approx: {str(apx):5s} | maxdiff: {maxdiff}')

## MLP
In the parameter initialisation, we use non-standard values to see their effect; otherwise, for example, zeros can mask out any incorrect values.

In [10]:
embed_dim = 10
hidden_dim = 200

g = torch.Generator().manual_seed(20240824)
C = torch.randn((vocab_size, embed_dim),  generator=g)

# hidden layer
fan_in = embed_dim*block_size # we concat multiple C's to feed into hidden layer
W1 = torch.randn((fan_in, hidden_dim), generator=g) * (5/3 / fan_in**0.5)
b1 = torch.randn(hidden_dim,           generator=g) * 0.1 # experiment
# output layer
W2 = torch.randn((hidden_dim, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size,               generator=g) * 0.1 # experiment with non-zero

# batch normalisation 1D layer placed after hidden layer, hence dim=hidden_dim
bn_gamma = torch.randn((1, hidden_dim),    generator=g) * 0.1 + 1.0
bn_bias = torch.randn((1, hidden_dim),     generator=g) * 0.1
#bn_running_mean = torch.zeros((1, hidden_dim))
#bn_running_std = torch.ones((1, hidden_dim))

# the above are initialised with non-standard values to magnify any incorrect values

parameters = [C, W1, W2, b2, bn_gamma, bn_bias]
print('total params: ', sum([p.nelement() for p in parameters]))
for p in parameters:
  p.requires_grad = True

total params:  12097


## training - extended version
We expand the forward pass into step by step calculations so that we can manually calculate the gradient step by step as well. For Cross Entropy loss function, see [pyTorch doco](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss).

We don't call the loss.backward(). Instead, we will do it manually.

In [8]:
# understanding tensor.values, which only works on sparse tensor
t = torch.randn((2,3))
sparse_tensor = t.max(dim=1, keepdim=True)
t, '-----------', sparse_tensor, '------------', sparse_tensor.values

(tensor([[-2.0463, -0.0297,  1.5362],
         [ 0.7847,  0.8150,  0.7719]]),
 '-----------',
 torch.return_types.max(
 values=tensor([[1.5362],
         [0.8150]]),
 indices=tensor([[2],
         [1]])),
 '------------',
 tensor([[1.5362],
         [0.8150]]))

In [11]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

  # construct mini-batch:
  # generate a list of random indices, length of list if batch_size
  ix = torch.randint(low=0, high=X_train.shape[0], size=(batch_size,), generator=g)
  xs = X_train[ix]  # (batch_size, block_size)
  ys = Y_train[ix]  # (batch_size)

  ##################################
  # forward pass (expanded version)
  ##################################
  # embedding ---------------------------
  emb = C[xs] # (batch_size, block_size, hidden_dim)
  embcat = emb.view(emb.shape[0], -1)
  # hidden layer ------------------------
  h_prebn = embcat @ W1 + b1 # (batch_size, hidden_dim)
  # BN layer (expended version) ----------------------------
  #bn_mean = h_prebn.mean(dim=0, keepdim=True)
  bn_mean = 1/batch_size*h_prebn.sum(dim=0, keepdim=True)
  #bn_std = h_prebn.std(dim=0, keepdim=True)
  bn_diff = (h_prebn - bn_mean)
  bn_diff2 = bn_diff ** 2
  bn_var = 1/(batch_size-1) * bn_diff.sum(dim=0, keepdim=True) # Bessel's correction, divide by m-1 not m
  bn_varinv = (bn_var + 1e-5)**-0.5  # 1/sqrt(var+eps)
  x_hat = bn_diff * bn_varinv
  h_preact = bn_gamma * x_hat + bn_bias
  #with torch.no_grad():
  #  bn_running_mean = 0.999 * bn_running_mean + 0.001 * bn_mean
  #  bn_running_std = 0.999 * bn_running_std + 0.001 * bn_std
  # Non-linearity ----------------------
  h = torch.tanh(h_preact)  # (batch_size, hidden_dim)
  # output layer -----------------------
  logits = h @ W2 + b2 # (hidden_dim, vocab_size)
  # loss function (extended version) ----------------------
  #loss = F.cross_entropy(logits, ys)
  logit_maxes = logits.max(dim=1, keepdim=True).values  # (hidden_dim, 1)
  norm_logits = logits - logit_maxes # subtract max for numerical stability
  counts = norm_logits.exp()  # (batch_size, vocab_size)
  counts_sum = counts.sum(dim=1, keepdim=True)
  counts_sum_inv = counts_sum ** -1
  probs = counts * counts_sum_inv
  logprobs = probs.log()   # (batch_size, vocab_size)
  loss = -logprobs[range(batch_size), ys].mean()  # scalar

  ################
  # backward pass
  ################
  for p in parameters:
    p.grad = None
  for t in [logprobs, probs, counts_sum_inv, counts_sum, counts,
            norm_logits, logit_maxes, logits,
            h, h_preact, x_hat, bn_varinv, bn_var, bn_diff2, bn_diff, bn_mean,
            h_prebn, embcat, emb ]:
    t.retain_grad()

  loss.backward()

  ###############
  # update
  ###############
  # lr = 0.1 if i<100000 else 0.01
  # for p in parameters:
  #   p.data += -lr * p.grad

  # # tracking
  # lossi.append(loss)
  # if i%10000 == 0:
  #   print('%6d/%7d %2.10f' % (i, max_steps, loss))

  # if i>1000:
  break
print('%6d/%7d %2.10f' % (i, max_steps, loss))

     0/ 200000 4.0160999298


In [22]:
ys.shape

torch.Size([32])

## Exercise 1 - backward pass

### dlogprobs
logprobs is dimension (N, vocab_size) , where N = batch_size
$$loss = -\dfrac{1}{N}\sum_{i=1}^{N}logprobs_{i, y_i}$$
$$\dfrac{\delta loss}{\delta logprobs}=
\begin{cases}
-\dfrac{1}{N} & \quad \text{at positions }{i},{y_i}\\
0 & \quad \text{elsewhere}
\end{cases}
$$

In [34]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(batch_size), ys] = -1/batch_size
cmp('dlogprobs', dlogprobs, logprobs)

dlogprobs       | exact: True  | approx: True  | maxdiff: 0.0


### dlogprobs
$$logprobs = \ln(probs)$$
$$\dfrac{\delta logprobs}{\delta probs} = \dfrac{1}{probs}
$$

In [23]:
dlogprobs = 1/probs # omit the *dloss since dloss=1

In [33]:
cmp('dlogprobs', dlogprobs, logprobs)
dlogprobs.shape

dlogprobs       | exact: False | approx: False | maxdiff: 3598.58984375


torch.Size([32, 27])

In [30]:
import math
probs[0], logprobs[0], math.log(0.0066)

(tensor([0.0171, 0.0066, 0.0091, 0.0543, 0.0534, 0.0135, 0.0023, 0.2083, 0.0601,
         0.1704, 0.0005, 0.0032, 0.0210, 0.0032, 0.0233, 0.1039, 0.0073, 0.0038,
         0.0319, 0.0212, 0.0172, 0.0036, 0.0025, 0.0191, 0.1093, 0.0230, 0.0110],
        grad_fn=<SelectBackward0>),
 tensor([-4.0706, -5.0247, -4.7035, -2.9132, -2.9308, -4.3018, -6.0565, -1.5687,
         -2.8119, -1.7698, -7.5795, -5.7603, -3.8640, -5.7566, -3.7590, -2.2639,
         -4.9168, -5.5704, -3.4465, -3.8535, -4.0607, -5.6183, -5.9743, -3.9600,
         -2.2140, -3.7742, -4.5074], grad_fn=<SelectBackward0>),
 -5.0206856299497575)

In [32]:
logprobs.grad[0], dlogprobs[0]

(tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000, -0.0312,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000]),
 tensor([  58.5935,  152.1314,  110.3359,   18.4162,   18.7425,   73.8336,
          426.8828,    4.8003,   16.6409,    5.8695, 1957.6235,  317.4359,
           47.6570,  316.2625,   42.9069,    9.6209,  136.5681,  262.5268,
           31.3899,   47.1599,   58.0156,  275.4112,  393.1922,   52.4590,
            9.1519,   43.5641,   90.6864], grad_fn=<SelectBackward0>))